In [1]:
import glob
import os
import sys
 
try:
    sys.path.append(glob.glob('carla-0.9.10-py3.7-linux-x86_64.egg')[0])
except IndexError:
    pass
 
from collections import deque
import math
import numpy as np
from enum import Enum
import random
import networkx as nx

import carla
from carla import ColorConverter as cc

import collections
import datetime
import logging
import re
import weakref
import cv2
import time

import h5py
import threading
from queue import Queue 
from tqdm import tqdm
from agents.navigation.behavior_agent import BehaviorAgent

In [2]:
class Sensors(object):
    def __init__(self):
        self.camera = dict()

class Measurement(object):  
    def __init__(self):
        self.steer = None
        self.gas = None
        self.brake = None
        self.velocity = None
        self.rgb = None
        
class World(object):
    
    restarted = False

    def __init__(self, client, spawn_idx):
        self.client = client
        self.world = client.get_world()
        self._roles_names = ['hero']
        self.actor_role_name = random.choice(self._roles_names)
        self._sensors = Sensors()
        self._measurements = dict()
        self.spawn_idx = spawn_idx
        
        try:
            self.map = self.world.get_map()
        except RuntimeError as error:
            logging.error('RuntimeError: {}'.format(error))
            logging.error('  The server could not send the OpenDRIVE (.xodr) file:')
            logging.error('  Make sure it exists, has the same name of your town, and is correct.')
            sys.exit(1)
        self.player = None
        self.camera_manager = None
        self._actor_filter = 'mustang'
        self._gamma = 2.2
        self.restart()

    def restart(self):
        self.player_max_speed = 1.589
        self.player_max_speed_fast = 3.713
        # Keep same camera config if the camera manager exists.
        cam_index = self.camera_manager.index if self.camera_manager is not None else 0
        cam_pos_index = self.camera_manager.transform_index if self.camera_manager is not None else 0
        # Get a random blueprint.
        blueprint = random.choice(self.world.get_blueprint_library().filter(self._actor_filter))
        blueprint.set_attribute('role_name', self.actor_role_name)
        if blueprint.has_attribute('color'):
            color = random.choice(blueprint.get_attribute('color').recommended_values)
            blueprint.set_attribute('color', color)
        if blueprint.has_attribute('driver_id'):
            driver_id = random.choice(blueprint.get_attribute('driver_id').recommended_values)
            blueprint.set_attribute('driver_id', driver_id)
        if blueprint.has_attribute('is_invincible'):
            blueprint.set_attribute('is_invincible', 'true')
        # set the max speed
        if blueprint.has_attribute('speed'):
            self.player_max_speed = float(blueprint.get_attribute('speed').recommended_values[1])
            self.player_max_speed_fast = float(blueprint.get_attribute('speed').recommended_values[2])
        else:
            print("No recommended values for 'speed' attribute")
        # Spawn the player.
        if self.player is not None:
            spawn_point = self.player.get_transform()
            spawn_point.location.z += 2.0
            spawn_point.rotation.roll = 0.0
            spawn_point.rotation.pitch = 0.0
            self.destroy()
            self.player = self.world.try_spawn_actor(blueprint, spawn_point)
        while self.player is None:
            if not self.map.get_spawn_points():
                print('There are no spawn points available in your map/town.')
                print('Please add some Vehicle Spawn Point to your UE4 scene.')
                sys.exit(1)
            #spawn_points = self.map.get_spawn_points()
            #spawn_point = random.choice(spawn_points) if spawn_points else carla.Transform()
            spawn_point = self.map.get_spawn_points()[self.spawn_idx]
            self.player = self.world.try_spawn_actor(blueprint, spawn_point)
            
        self.camera_manager = CameraManager(self.player, self._gamma, 384, 160)
            
    def destroy_sensors(self):
        self.camera_manager.sensor.destroy()
        self.camera_manager.sensor = None
        self.camera_manager.index = None

    def destroy(self):
        actors = [
            self.camera_manager.sensor,
            self.player]
        for actor in actors:
            if actor is not None:
                actor.destroy()
    
    def get_measurements(self):
        control = self.player.get_control()
        vel = self.player.get_velocity()
        velocity = math.sqrt(vel.x ** 2 + vel.y ** 2 + vel.z ** 2)
        measurement = Measurement()
        round_dec = 4
        measurement.steer = round(control.steer,round_dec)
        measurement.gas = round(control.throttle,round_dec)
        measurement.brake = round(control.brake,round_dec)
        measurement.velocity = round(velocity,round_dec)
        measurement.command = None
        measurement.rgb = self.camera_manager.surface
        return measurement

In [3]:
class CameraManager(object):
  
    def __init__(self, parent_actor, gamma_correction, width = 640, height = 480):
        self.width = width
        self.height = height
        self.sensor = None
        self.surface = np.zeros(shape=(self.height, self.width, 3), dtype=np.uint8)
        self._parent = parent_actor
        
        bound_y = 0.5 + self._parent.bounding_box.extent.y
        Attachment = carla.AttachmentType
        self._camera_transforms = [
            (carla.Transform(carla.Location(x=1.6, z=1.7)), Attachment.Rigid),
        ]

        world = self._parent.get_world()
        bp_library = world.get_blueprint_library()
        bp = bp_library.find('sensor.camera.semantic_segmentation')
        bp.set_attribute('image_size_x', str(self.width))
        bp.set_attribute('image_size_y', str(self.height))
        if bp.has_attribute('gamma'):
            bp.set_attribute('gamma', str(gamma_correction))
        
        self.sensor = self._parent.get_world().spawn_actor(bp,self._camera_transforms[0][0],attach_to=self._parent,attachment_type=self._camera_transforms[0][1])     
        weak_self = weakref.ref(self)
        self.sensor.listen(lambda image: CameraManager._parse_sensor(weak_self, image, 0)) 
    
    @staticmethod
    def _parse_sensor(weak_self, image, sensor_index=0):
        self = weak_self()
        if not self:
            return
        self.surface = CameraManager._parse_image(weak_self, image, sensor_index)
        
    @staticmethod
    def _parse_image(weak_self, image, sensor_index=0):
        self = weak_self()
        if not self:
            return
        image.convert(cc.CityScapesPalette)
        array = np.frombuffer(image.raw_data, dtype=np.dtype("uint8"))
        array = np.reshape(array, (image.height, image.width, 4))
        array = array[:, :, :3]
        array = array[:, :, ::-1]
        return array.copy()

In [4]:
def worker_recording(in_q):
    ######################DATASET###########################
    today = datetime.date.today()
    dataset_path = './dataset/'
    sample_time = 5
    dataset_num = 1
    batch = 400

    dataset_measurements = []
    dataset_rgb = []
    SPECIAL_TYPE = np.dtype([
        ("steer", np.float32),
        ("throttle", np.float32),
        ("brake", np.float32),
        ("velocity", np.float32),
        ("command", np.int8)
    ])
    logging.info('🐧🐧🐧 Record Start!🐧🐧🐧')

    for dataset_count in range(0, dataset_num):
        logging.info('🐧🐧🐧 New Dataset {}!🐧🐧🐧'.format(dataset_count))

        for data_count in tqdm(range(0, batch), desc ="Data"):
            observations = in_q.get()
            dataset_measurements.append((observations.steer, observations.gas, observations.brake, observations.velocity, observations.command))
            dataset_rgb.append(observations.rgb)
            time.sleep(sample_time)    

        hdf = h5py.File(dataset_path + "carla-dataset-{}_{}.h5".format(today, dataset_count),'w')
        hdf.create_dataset("measurements", data=np.array(dataset_measurements, dtype=SPECIAL_TYPE) , dtype=SPECIAL_TYPE)
        hdf.create_dataset("rgb", data=dataset_rgb, dtype='uint8')
        dataset_measurements = []
        dataset_rgb = []
        in_q.queue.clear()
        hdf.close()   

    logging.info('🐧🐧🐧 Record End!🐧🐧🐧')

In [5]:
def worker_agent(out_q):
    client = carla.Client('127.0.0.1', 2000)
    client.set_timeout(5.0)
    client.load_world('Town02')
    world = World(client, 0) 
    logging.info('Starting test {}'.format(world.actor_role_name))
    checkpoints = [36, 17, 5, 56]
    checkpoints_idx = 0
    
    #####################AGENT############################
    agent = BehaviorAgent(world.player, ignore_traffic_light=True, behavior='normal')
    spawn_points = world.map.get_spawn_points()
    destination = spawn_points[checkpoints_idx].location
    agent.set_destination(agent.vehicle.get_location(), destination, clean=True)
    #################################################

    while not stop_threads:
        agent.update_information(world)
        speed_limit = world.player.get_speed_limit()
        agent.get_local_planner().set_speed(speed_limit)
        control = agent.run_step()
        world.player.apply_control(control)
        observations = world.get_measurements()
        observations.command = agent.direction.value
        out_q.put(observations) 
        
        if len(agent.get_local_planner().waypoints_queue) == 0:
            checkpoints_idx = checkpoints_idx + 1
            if checkpoints_idx >= len(checkpoints):
                checkpoints_idx = 0
            spawn_point = spawn_points[checkpoints[checkpoints_idx]]
            agent.set_destination(agent.vehicle.get_location(), spawn_point.location, clean=True)
        
        #cv2.imshow('image',observations.rgb)
        #cv2.waitKey(1)
        #sys.stdout.write("\r" + "measurements steer: {}, gas: {}, vel: {}, command: {}".format(observations.steer, observations.gas, observations.velocity, observations.command))
        #sys.stdout.flush()    

In [12]:
log_level = logging.INFO
logging.basicConfig(format='%(levelname)s: %(message)s', level=log_level)
world = None
stop_threads = False
q = Queue()

try:
    logging.info('🐧🐧🐧 Start 🐧🐧🐧')
    thread_agent = threading.Thread(target=worker_agent, args =(q, ))
    thread_record = threading.Thread(target=worker_recording, args =(q, ))
    thread_agent.start()
    thread_record.start()
    while True:
        time.sleep(.1)
    logging.info('🐧🐧🐧 Fin!🐧🐧🐧')
    
except KeyboardInterrupt:
    cv2.destroyAllWindows()
    stop_threads = True
    thread_record.join()
    thread_agent.join()
    logging.info('\nCancelled by user. Bye!')


INFO: 🐧🐧🐧 Start 🐧🐧🐧
INFO: 🐧🐧🐧 Record Start!🐧🐧🐧
INFO: 🐧🐧🐧 New Dataset 0!🐧🐧🐧


Data:  30%|███       | 61/200 [01:03<02:19,  1.00s/it]INFO: Starting test hero
No recommended values for 'speed' attribute
deque index out of range

Data:  31%|███       | 62/200 [01:04<02:18,  1.00s/it]

Data:  32%|███▏      | 63/200 [01:05<02:17,  1.00s/it]

Data:  32%|███▏      | 64/200 [01:06<02:16,  1.01s/it]

Data:  32%|███▎      | 65/200 [01:07<02:15,  1.01s/it]

Data:  33%|███▎      | 66/200 [01:08<02:14,  1.01s/it]

Data:  34%|███▎      | 67/200 [01:09<02:13,  1.01s/it]

Data:  34%|███▍      | 68/200 [01:10<02:12,  1.01s/it]

Data:  34%|███▍      | 69/200 [01:11<02:11,  1.01s/it]

Data:  35%|███▌      | 70/200 [01:12<02:10,  1.01s/it]

Data:  36%|███▌      | 71/200 [01:13<02:09,  1.01s/it]

Data:  36%|███▌      | 72/200 [01:14<02:08,  1.01s/it]

Data:  36%|███▋      | 73/200 [01:15<02:07,  1.01s/it]

Data:  37%|███▋      | 74/200 [01:16<02:06,  1.01s/it]

Data:  38%|███▊      | 75/200 [01:17<02:05,  1

KeyboardInterrupt: 